In [4]:
!pip install requests-html
!pip install bs4
!pip install lxml[html_clean]

In [5]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from collections import deque
from urllib.parse import urlsplit
import pandas as pd
from requests_html import HTMLSession
import threading
import time
from tqdm import tqdm

In [6]:
#user_url = "https://www.thapar.edu/sitemap.xml"
#url = ["https://www.thapar.edu"]
EMAIL_REGEX = r"""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])"""

In [12]:
def get_urls_of_xml(xml_url):
    r = requests.get(xml_url, verify=False)
    xml = r.text
    # Use the lxml XML parser
    soup = BeautifulSoup(xml, 'lxml-xml')

    links_arr = []
    # Use find_all instead of findAll as recommended by the DeprecationWarning
    for link in soup.find_all('loc'):
        linkstr = link.getText('', True)
        links_arr.append(linkstr)

    return links_arr



links_data_arr = get_urls_of_xml("https://thapar.edu/sitemap.xml")
#print(links_data_arr)

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'thapar.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [9]:
session = HTMLSession()

In [ ]:
email=set()
for i in tqdm(links_data_arr):
        #requests.get(f'{i}', verify=False)
        r = session.get(i)
        for re_match in re.finditer(EMAIL_REGEX, r.html.raw_html.decode()):
            email.add(((re_match.group())).replace("-",""))

In [16]:
df = pd.DataFrame(email)
df.to_csv('Emails.csv', index=False)